In [3]:
#imports
import numpy as np
import torch.optim as optim
import math
import argparse
import sys

from torch import nn
from dataloader import *
from models import *
from helper import *
from trainAndtest import *

In [4]:
percentage = 0.05

X_train, X_val, train_nc, X_test = DataProcessing(float(percentage))
train_loader = DataLoader(X_train, batch_size=16, shuffle=True, num_workers=16)
val_loader = DataLoader(X_val, batch_size=16, shuffle=True, num_workers=16)
test_loader = DataLoader(X_test, batch_size=16, shuffle=True, num_workers=16)

=============== Loading Data ===============


In [6]:
## 
mname = None
train_models = []
best_score = 999999
best_threshold = [0.22825835302337297,0.2241922165776087,0.26945796358407853,0.2546193993822431,0.10096654458404705,0.1570157025676272,0.18412963345438044,0.2023645493433312,0.20194953713647665,0.3023475291366101,0.20763791557387953,0.07203177923252294,0.14270117753775374,0.32631954254330175,0.2292770860903799,0.16027637330660788,0.20264818991555558]
print(len(best_threshold))

17


In [ ]:
for mt in ["resnet50", "densenet161", "resnet152"]:
    model_type = mt

    model = get_from_models(model_type, 17, not(mname==None), mname)
    model = model.cuda()

    # Created the loss function and optimizer
    loss_fn = nn.MultiLabelSoftMarginLoss().cuda()
    optimizer = optim.SGD([
        {'params': [param for name, param in model.named_parameters() if name[-4:] == 'bias']},
        {'params': [param for name, param in model.named_parameters() if name[-4:] != 'bias'],
         'weight_decay': 1e-4}
    ], lr=1e-2, momentum=0.9, nesterov=True)


    for epoch in range(1, 5):
        train(model, epoch, train_loader, optimizer, loss_fn)
        torch.save(model.state_dict(),"./ensembler/{}.pth".format(model_type))
        v_score, threshold, ff2 = validate(model, val_loader, loss_fn)

        if v_score < best_score:
            best_model = model
            best_threshold = threshold
            best_score = v_score
            # saveCreds(save = True, threshold = best_threshold, v_score = best_score)
            # torch.save(model.state_dict(),"./{}_train.pth".format("best"))
        

### Ensembler Predictions


In [ ]:
ensembleResnetModelsPredictions(test_loader, None, best_threshold, X_test, "Prediction", ["resnet50"])

In [ ]:
ensembleResnetModelsPredictions(test_loader, None, best_threshold, X_test, "Prediction", ["densenet161"])

In [ ]:
ensembleResnetModelsPredictions(test_loader, None, best_threshold, X_test, "Prediction", ["resnet152"])

In [7]:
ensembleResnetModelsPredictions(test_loader, None, best_threshold, X_test, "Prediction", ["resnet50","resnet152"])

====== Starting Prediction Model Enesembling ======
load model ./ensembler/resnet50.pth
Getting resnet50
pretrained
Done 0.0%
Done 7.90904597133%
Done 15.8180919427%
Done 23.727137914%
Done 31.6361838853%
Done 39.5452298566%
Done 47.454275828%
Done 55.3633217993%
Done 63.2723677706%
Done 71.181413742%
Done 79.0904597133%
Done 86.9995056846%
Done 94.908551656%
load model ./ensembler/resnet152.pth
Getting resnet152
pretrained
Done 0.0%
Done 7.90904597133%
Done 15.8180919427%
Done 23.727137914%
Done 31.6361838853%
Done 39.5452298566%
Done 47.454275828%
Done 55.3633217993%
Done 63.2723677706%
Done 71.181413742%
Done 79.0904597133%
Done 86.9995056846%
Done 94.908551656%
Main Prediction [9.7164738e-01 1.6690182e-03 1.7870545e-02 1.3087963e-03 1.1193042e-03
 9.5225924e-01 8.7388253e-05 1.1386208e-03 7.0863950e-01 9.8594099e-01
 2.7168173e-02 3.1191852e-02 9.9682981e-01 9.7724664e-01 4.0940051e-03
 5.5671078e-03 1.0972444e-01], prediction [4.0392828e-01 9.3143592e-03 4.3006573e-02 3.0148763e-0